<a href="https://colab.research.google.com/github/Auxilus08/Compiler-Design/blob/main/Cd_practical3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parser Construction

In [ ]:
import sys

grammer = {}
non_terminals = set()
first_sets = {}

def parse_grammer(input_lines):
  for line in input_lines:
    line = line.strip()
    if not line or line.startswith('#'):
      continue
    head, _, str_productions = line.partition(':')
    head = head.strip()

    if not head.isupper():
      print(f" Non terminal {head} is not uppercase.")

    non_terminals.add(head)

    if head not in grammer:
      grammer[head] = []

    productions = str_productions.split('|')
    for production in productions:
      production = production.strip()
      grammer[head].append(production)

In [ ]:
def calculate_first_for_sequence(sequence):
  first = set()

  if not sequence or sequence[0] == 'Epsilon':
    return {'Epsilon'}
  can_produce_epsilon = True

  for symbol in sequence:
    if symbol not in non_terminals:
      first.add(symbol)
      can_produce_epsilon = False
      break

    first_of_symbol = first_sets[symbol]
    first.update(first_of_symbol - {'Epsilon'})

    if 'Epsilon' not in first_of_symbol:
      can_produce_epsilon = False
      break

  if can_produce_epsilon:
    first.add('Epsilon')

  return first